In [1]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import numpy as np
from PowerModel import opt_power
import matplotlib.pyplot as plt

In [ ]:
pv_attr = pd.read_csv('./data/vre/desert_solar.csv',index_col=['lon','lat'])
pv_prof = pd.read_pickle('./data/vre/desert_solar_cfprof.pkl') # this file is too large, please contact lead contact

we_attr = pd.read_csv('./data/vre/desert_we.csv',index_col=['lon','lat'])
we_prof = pd.read_pickle('./data/vre/desert_wind_cfprof.pkl')# this file is too large, please contact lead contact


In [3]:

lon = 22.75
lat = 19.75

res = opt_power(pv_cap=pv_attr.at[(lon,lat),'cap_mid'],
                pv_cf=pv_prof[(lon,lat)],
                we_cap=we_attr.at[(lon,lat),'cap_mid'],
                we_cf=we_prof[(lon,lat)])

Set parameter Method to value 2
Set parameter Crossover to value 0


In [ ]:
vis_df = pd.DataFrame.from_dict(res['ts']) * 100


In [16]:
idx = np.arange(8760) // 24

# 使用 groupby 和 sum 进行分组求和
vis_df_day = vis_df.groupby(idx).sum()

In [17]:
vis_df_day['sto_netdis'] = vis_df_day['sto_dischar']-vis_df_day['sto_char']
vis_df_day['sto_netchar'] = vis_df_day['sto_char']-vis_df_day['sto_dischar']

vis_df_day.loc[vis_df_day['sto_netdis']<0,'sto_netdis'] = 0
vis_df_day.loc[vis_df_day['sto_netchar']<0,'sto_netchar'] = 0

In [ ]:
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['font.size'] = 25


color = {
    'onshore':'#499BC0',
    'offshore':'#8FDEE3',
    'upv':'#F78779',
    'dpv':'#FDD786'
}

fig,ax = plt.subplots(figsize=(28,8),dpi=600)


vis_df_day.plot(kind='area',
                ax=ax,
                y=['we_output','pv_output','sto_netdis'],
                label=['Wind','Solar','Storage discharge'],
                color={'Solar':'#ff2056',
                       'Wind':'#2b7fff',
                       'Storage discharge':'#4d179a'},
                lw=0.1)

ax.fill_between(range(365),
                24*np.ones(365),
                vis_df_day['sto_netchar']+24*np.ones(365),
                alpha=0.5,
                facecolor='none',
                hatch='//',
                label='Storage charge')

ax.axhline(y=24)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)


ax.set_xticks(range(0,365,12),range(0,365,12))

ax.set_xlim(0,365)
ax.set_ylim(0,35)

ax.minorticks_on()

ax.tick_params(axis='y', 
               which='major', 
               length=8, 
               color='black')

ax.tick_params(axis='y', 
               which='minor', 
               length=4, 
               color='black')

ax.tick_params(axis='x', 
               which='minor', 
               length=4, 
               color='black')

ax.tick_params(axis='x', 
               which='major', 
               length=8, 
               color='black')

ax.legend(edgecolor='black',
          ncol=4,
          loc='upper center')

ax.set_xlabel('Time (day)')
ax.set_ylabel('Output (GW)')


plt.savefig('./data/fig/Vis/wss_combine_yr.jpg',bbox_inches='tight')

plt.show()

In [19]:
season_data = {}


season_data['Spring'] = vis_df.iloc[120*24:121*24].copy(deep=True).reset_index()
season_data['Summer'] = vis_df.iloc[220*24:221*24].copy(deep=True).reset_index()
season_data['Autumn'] = vis_df.iloc[323*24:324*24].copy(deep=True).reset_index()

season_data['Winter'] = vis_df.iloc[340*24:341*24].copy(deep=True).reset_index()


In [ ]:
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['font.size'] = 25

fig,axes = plt.subplots(1,4, figsize=(28,6),dpi=600,sharey=True)


# 绘制每种能源的数据
for idx,ses in enumerate(['Spring','Summer','Autumn','Winter']):
    ax = axes[idx]
    
    season_data[ses].plot(kind='area',
                          ax=ax,
                          y=['we_output','pv_output','sto_dischar'],
                          label=['Wind','Solar','Storage discharge'],
                          color={'Solar':'#ff2056',
                                 'Wind':'#2b7fff',
                                 'Storage discharge':'#4d179a'},
                          lw=0.1,
                          legend=False)
    
    ax.fill_between(range(24),
                    np.ones(24),
                    season_data[ses]['sto_char']+np.ones(24),
                    alpha=0.5,
                    facecolor='none',
                    hatch='//',
                    label='Storage charge')
    
    ax.minorticks_on()

    ax.tick_params(axis='y', 
                which='major', 
                length=6, 
                color='black')
    
    ax.tick_params(axis='y', 
                which='minor', 
                length=3, 
                color='black')

    ax.tick_params(axis='x', 
                    which='minor', 
                    length=2, 
                    color='blue')

    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    ax.set_xlabel('Hour of day')
    ax.set_xlim(0,23)
    ax.set_ylim(0,4)
    
    ax.axhline(y=1,color='black')
    
    ax.set_title(ses)
    
    if idx == 0:
        ax.set_ylabel('Output (GW)')

    ax.set_xticks(range(0,24,4),range(0,24,4))
    
plt.subplots_adjust(wspace=0.1)

plt.savefig('./data/fig/Vis/wss_combine_day.jpg',bbox_inches='tight')

plt.show()